# Web scraping for houses dataset
In this notebook, we will create a dataset of houses found from [Funda](https://www.funda.nl/) (Dutch real-estate website). In order to do this, we need to program a web bot to retrieve all the information for us. We will use a combination of [Selenium](https://selenium-python.readthedocs.io/) and [Beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) for this.

This notebook is part of my House Price series in which we create a dataset, train a prediction model, and deploy the model and an accompanying web app.

## Imports

In [378]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

## Getting data

First, we need to give our bot acces to the internet. For this we use Selenium. Selenium is a python package that automates web browser interaction. 

First we start our webdriver

In [397]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
url="https://www.funda.nl/koop/heel-nederland/"
driver.get(url)

A screen must have popped up showing an empty browser. This is the browser that will be used by our bot. 

Now we do some initializations. These lists are all predefined in order for our information to be appended to it.

In [398]:
adresses = [] 
cities = []
prices = [] 
lot_sizes = []
build_years = []
build_types = []
house_types = []
roofs = []
rooms = []
toilets = []
floors = []
energylabels = []
positions = []
gardens = []

true_features = ['Soort woonhuis', 'Soort bouw', 'Bouwjaar', 'Soort dak', 'Aantal kamers', 'Aantal badkamers', 'Aantal woonlagen', 'Energielabel', 'Ligging', 'Tuin']
arrays = [house_types, build_types, build_years, roofs, rooms, toilets, floors, energylabels, positions, gardens]

Now the main part starts, retrieving the data. Our bot can be told to go to a certain URL and retrieve information, but the information our bot retrieves is _ALL_ the HTML that is on the webpage. In order to make sense of this, we use Beautifulsoup. Beautifulsoup can do quick searches in HTML and XML data. 

In order to tell beautifulsoup which data to find, we need to explicitly mention the type of element, name of the element's class or element attributes. This causes  a lot of searching and trial and error, in order to find the correct class names and get the data out in the form that you want.

Funda gives results of houses like the following, showing the adress, price, size and number of rooms:

![img](assets/Funda.png)

The HTML location of this information can be found, when inspecting the page. We can, for example, see that the data on the price is stored inside a <font color='red'>'data-test-id'</font> element with the class: <font color='red'>'price-sale'</font>.

![img](assets/FundaPrice.png)

In beautifulsoup, this can be retrieved using:

```python
price = text.find('span', attrs={'class':'search-result-price'})
```

For more information about class names and search strategies in beautifulsoup, I suggest reading an in-depth web scraping tutorial like [this one](https://www.dataquest.io/blog/web-scraping-beautifulsoup/).

In our case, retrieving the data about the houses is a tad harder. This is because the search results do not give all data on the houses. More data is given on the webpage of each house listing. Our approach is therefore to navigate to this listing (retrieve URL first) and do the same process of retrieving data there too. 

See if you can understand this part of the code too :-)

> IMPORTANT: Before running this code (or code like it that you have written yourself), make sure that you navigate to your URL in the pop up browser window first. Sometimes here you need to do a captcha or other 'prove you are not a bot' test. The bot itself cannot move past these screens.

In [399]:

url = "https://www.funda.nl/zoeken/koop/?selected_area=%5B%22nl%22%5D" # The URL with the data you want to extract
for j in tqdm(range(650,666)): # In this case we extract data from 500 pages of results
    if j==1:
        driver.get(url)
    else:
        driver.get(url+'&search_result='+str(j)) # Add string defining the page of the search results

    content = driver.page_source
    soup = BeautifulSoup(content) 
    for a in soup.findAll('div',href=False, attrs={'data-test-id': 'search-result-item'}): # Loop over all individual house results
        # Find initial data
        address = a.find('h2', attrs={'data-test-id':'street-name-house-number'})
        city = a.find('div', attrs={'data-test-id':'postal-code-city'})
        price = a.find('p', attrs={'data-test-id':'price-sale'})
        lot_size = a.find('li', attrs={'class':'mr-4 flex flex-[0_0_auto]'})

        # Find URL of listing and extract HTML data
        listing_url = a.find('a', href=True, attrs={'data-test-id':'object-image-link'})
        driver.get(listing_url['href'])
        content_full = driver.page_source
        soup_full = BeautifulSoup(content_full) 
        
        # Find list of house characteristics
        characteristics = soup_full.findAll('dl', attrs={'class':'object-kenmerken-list'})

        feature_names = []
        features = []

        loop_range = np.arange(len(characteristics))
        loop_range = loop_range[np.arange(len(loop_range))!=2]
        loop_range = loop_range[np.arange(len(loop_range))!=4]
        
        # For each characteristic, retrieve the corresponding value
        for i in loop_range:
            for feature in characteristics[i].findAll('dt'):
                feature_names.append(list(filter(None, feature.text.split('\n')))[0])

            for feature in characteristics[i].findAll('dd'):
                try:
                    features.append(list(filter(None, feature.text.split('\n')))[0])
                except: 
                    pass
        
        # Add data to corresponding predefined list        
        for name, array in zip(true_features, arrays):
            if name in feature_names:
                array.append(features[feature_names.index(name)])
            else:
                array.append(np.nan)        
            
            # Append initial data to lists
        adresses.append(address.text.split("\n")[1].lstrip())
        prices.append(price.text.split('\n')[1].lstrip().replace(' k.k.',"").replace('€ ',"").replace('.',""))
        cities.append(' '.join(city.text.split('\n')[1].split(' ')[14:]))
        lot_sizes.append(lot_size.text.split('\n')[1].lstrip().replace(' m²',""))


100%|██████████| 16/16 [07:46<00:00, 29.18s/it]


In [299]:
print(feature_names)

['Vraagprijs', '    Vraagprijs per m²', 'Aangeboden sinds', 'Status', 'Aanvaarding', 'Bijdrage VvE', 'Soort appartement', 'Soort bouw', 'Bouwjaar', 'Soort dak', 'Aantal kamers', 'Aantal badkamers', 'Badkamervoorzieningen', 'Aantal woonlagen', 'Voorzieningen', 'Energielabel', 'Isolatie', 'Verwarming', 'Warm water', 'Cv-ketel', 'Ligging', 'Balkon/dakterras', 'Schuur/berging', 'Soort parkeergelegenheid', 'Inschrijving KvK', 'Jaarlijkse vergadering', 'Periodieke bijdrage', 'Reservefonds aanwezig', 'Onderhoudsplan', 'Opstalverzekering']


In [400]:
df = pd.DataFrame({'Address': adresses,
                   'City': cities,
                   'Price': prices,
                   'Lot size (m2)': lot_sizes,
                   'Build year': build_years,
                   'Build type': build_types,
                   'House type': house_types,
                   'Roof': roofs,
                   'Rooms': rooms,
                   'Toilet': toilets,
                   'Floors': floors,
                   'Energy label': energylabels,
                   'Position': positions,
                   'Garden': gardens}) 
df.size

3360

After this has run, we can combine the lists into a DataFrame. Shown below, is a partially retrieved dataset of a 1000 entries. I retrieved data in chunks of 50-100 pages, as extracting 500 pages of data in one go took a long time.

In [272]:
print('Dataset consists of information on ' + str(len(df)) + ' houses')

Dataset consists of information on 45 houses


## Saving data
Finally, we save the data. In this case, this was the data from page 401-500.

In [401]:
df.to_csv('house_data_650-666.csv', index=False, encoding='utf-8')

## Merging the files
Here is the ETL steps to bring the different excel sheets together.


In [413]:
df1=pd.read_csv('house_data_1-130.csv')
df2=pd.read_csv('house_data_130-150.csv')
df3=pd.read_csv('house_data_150-180.csv')
df4=pd.read_csv('house_data_180-200.csv')
df5=pd.read_csv('house_data_200-240.csv')
df6=pd.read_csv('house_data_240-280.csv')
df7=pd.read_csv('house_data_280-320.csv')
df8=pd.read_csv('house_data_320-360.csv')
df9=pd.read_csv('house_data_360-427.csv')
df10=pd.read_csv('house_data_427-470.csv')
df11=pd.read_csv('house_data_470-520.csv')
df12=pd.read_csv('house_data_520-577.csv')
df13=pd.read_csv('house_data_577-614.csv')
df14=pd.read_csv('house_data_614-641.csv')
df15=pd.read_csv('house_data_641-650.csv')
df16=pd.read_csv('house_data_650-666.csv')



In [418]:
dffinal=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16], axis=0)

In [420]:
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9987 entries, 0 to 239
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address        9987 non-null   object 
 1   City           9987 non-null   object 
 2   Price          9987 non-null   object 
 3   Lot size (m2)  9987 non-null   float64
 4   Build year     9816 non-null   object 
 5   Build type     9974 non-null   object 
 6   House type     7940 non-null   object 
 7   Roof           9653 non-null   object 
 8   Rooms          9975 non-null   object 
 9   Toilet         9975 non-null   object 
 10  Floors         9975 non-null   object 
 11  Energy label   9974 non-null   object 
 12  Position       9443 non-null   object 
 13  Garden         8302 non-null   object 
dtypes: float64(1), object(13)
memory usage: 1.1+ MB


In [ ]:
dffi=dffinal.drop_duplicates()


In [ ]:
dffi.to_csv('house_alldata.csv', index=False, encoding='utf-8')